In [56]:
from pathlib import Path
import numpy as np
from collections import deque
from copy import copy

np.set_printoptions(threshold = np.inf, linewidth = np.inf)

In [57]:
data = Path('Day23.txt').read_text().splitlines()
data = np.array([[c for c in line] for line in data])

h, w = data.shape

In [58]:
to_visit = deque([[set(), 0, 1]])

neighbors = [[1, 0], [-1, 0], [0, 1], [0, -1]]

max_dist = 0
best_path = None

while to_visit:
    visited, i, j = to_visit.popleft()

    if i == h -1:
        if len(visited) > max_dist:
            max_dist = len(visited)
            best_path = visited

    visited = copy(visited)
    visited.add((i,j))

    if data[i,j] == '>':
        if (i, j+1) not in visited:
            to_visit.append([visited, i, j+1])
    elif data[i,j] == 'v':
        if (i+1, j) not in visited:
            to_visit.append([visited, i+1, j])
    else:
        for di, dj in neighbors:
            i2 = i + di
            j2 = j + dj

            if 0 <= i2 < h and 0<= j2 < w and data[i2, j2] != '#' and (i2,j2) not in visited:
                to_visit.append([visited, i2, j2])

print(max_dist)

2318


# Part 2

In [59]:
forks = []

for i in range(h):
    for j in range(w):
        if data[i,j] != '#':
            valid_neighbors = 0

            for di, dj in neighbors:
                i2 = i + di
                j2 = j + dj

                if 0 <= i2 < h and 0 <= j2 < w and data[i2, j2] != '#':
                    valid_neighbors += 1
            
            if valid_neighbors > 2:
                forks.append([i, j])

In [60]:
neighbors = [[1, 0], [-1, 0], [0, 1], [0, -1]]

def follow_to_fork(start_i, start_j):
    to_visit = deque([[start_i, start_j, 0]])
    visited = set()

    end_forks = []

    while to_visit:
        i, j, steps = to_visit.popleft()

        visited.add((i, j))

        valid_paths = [[i2, j2] for di,dj in neighbors if 
            0 <= (i2:=i+di) < h and 0 <= (j2:=j+dj) < w and data[i2, j2] != '#' and (i2, j2) not in visited]
        
        if len(valid_paths) == 1 or (i == start_i and j == start_j):
            for [i2, j2] in valid_paths:
                to_visit.append([i2, j2, steps+1])
        else:
            end_forks.append([i, j, steps])

    return end_forks

In [61]:
paths = {}

for fork in [[0, 1]] + forks:
    end_forks = follow_to_fork(fork[0], fork[1])

    paths[tuple(fork)] = {(i, j): steps for i,j,steps in end_forks}

In [62]:
to_visit = deque([[0, 1, set(), 0]])

max_steps = 0

while to_visit:
    i, j, visited, steps = to_visit.popleft()

    if i == h-1:
        max_steps = max(max_steps, steps)
        continue

    visited = copy(visited)
    visited.add((i, j))

    for (i2, j2), steps2 in paths[i,j].items():
        if (i2, j2) not in visited:
            to_visit.append([i2, j2, visited, steps + steps2])

print(max_steps)

6426
